In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM  

In [3]:
tokenizer = AutoTokenizer.from_pretrained("llmware/bling-1.4b-0.1")  



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model = AutoModelForCausalLM.from_pretrained("llmware/bling-1.4b-0.1")  



In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

bs4_strainer = bs4.SoupStrainer(class_=("main-content"))
loader = WebBaseLoader(
    web_paths=("https://deltek.com/en","https://www.deltek.com/en/about/contact-us", "https://www.deltek.com/en/small-business", "https://www.deltek.com/en/customers",
               "https://www.deltek.com/en/support", "https://www.deltek.com/en/partners"),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()



text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
all_splits_text = [split.page_content for split in all_splits]



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from sentence_transformers import SentenceTransformer
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings


# doc_embeddings = SentenceTransformer('all-MiniLM-L6-v2').encode(all_splits_text)
# query = "Who trusts Deltek?"
# query_embeddings = SentenceTransformer('all-MiniLM-L6-v2').encode(query)
# # print(doc_embeddings.shape)
# pairs = zip(all_splits_text, doc_embeddings)

# # vectorstore = FAISS.from_embeddings(pairs, SentenceTransformer('all-MiniLM-L6-v2'))



model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    # query_instruction="What is Deltek?"
)

query = "Who trusts Deltek?"



In [7]:
# from sklearn.metrics.pairwise import cosine_similarity

# results = cosine_similarity(doc_embeddings, query_embeddings.reshape(1,-1)).reshape((-1,))
# k = 4
# ixs = results.argsort()
# ixs = ixs[::-1]

# relevant_docs = []
# results2 = []
# for i in ixs:
#     relevant_docs.append(all_splits[i].page_content)
#     results2.append(results[i])
# relevant_docs = relevant_docs[:k]
# formatted_docs = "\n\n".join(doc for doc in relevant_docs)

db = FAISS.from_documents(all_splits, embeddings)
query = "Who trusts deltek?"
relevant_docs = db.similarity_search(query)
formatted_docs = '\n'.join(doc.page_content for doc in relevant_docs)
print(formatted_docs)


*






Message












Yes, I would like to receive email about news, products and research from Deltek.



Submit


If you need support for your Deltek solution, please open a case in the Deltek Support Center.


Your personal information, including your email address, will be held in the strictest of confidence and will never be shared with anyone as stipulated in our Privacy Policy.
*



State
Alabama
Alaska
Arizona 
Arkansas 
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming












Yes, I would like to receive email about news, products and research from Deltek.




Su

In [8]:
context = formatted_docs
print(len(relevant_docs))

4


In [9]:
entries = {"context": context, 
          "query":query}


In [10]:
new_prompt = "<human>: " + entries["context"] + "\n" + entries["query"] + "\n" + "<bot>:"

inputs = tokenizer(new_prompt, return_tensors="pt")  
start_of_output = len(inputs.input_ids[0])

In [11]:
outputs = model.generate(
        inputs.input_ids.to("cpu"),
        attention_mask=inputs.attention_mask.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.3,
        max_new_tokens=100,
        )


In [12]:
output_text = tokenizer.decode(outputs[0][start_of_output:], skip_special_tokens=True)
print(output_text)

 Deltek is trusted by the most important customers in the industry.
